# Gensim 패키지
- Python으로 작성된 오픈 소스 라이브러리로, 자연어 처리와 관련된 다양한 기능을 제공한다.
- 주요 기능
    - **Word Embeddings**
        - word2vec, fastext, doc2vec 등 다양한 word embedding 모델을 제공
    - **토픽 모델링 (Topic Modeling)**
        - LDA등 문장의 주제를 파악하는 모델 제공
    - **텍스트/word 유사도 계산**
    - **문서 군집화**
        - 비슷한 주제의 문서들을 군집화.
    - 다양한 dataset과 pretrained model 제공
        - https://github.com/piskvorky/gensim-data
- https://radimrehurek.com/gensim/

요즘은 Gensim보다는 Hugging Face 많이 씁니다. 그렇게 중요한 패키지는 아니지만, word2vec을 실습해보자는 의미에서 가볍게 보아요.

## 설치
- `pip install gensim`

In [1]:
%pip install gensim --upgrade

Note: you may need to restart the kernel to use updated packages.


In [2]:
import gensim

# Word2Vec 학습

- gensim.models.Word2Vec
- 주요 파라미터
    - sentences
        -  학습에 사용할 문서의 리스트. 각 문서의 단어들을 리스트로 묶고 그 문서들을 리스트로 묶은 중첩 리스트.
        - 예시: \[\['word1', 'word2', 'word3'], \['word4', 'word5']]
    - vector_size
        -  embedding vector 크기. 기본값: 100
    - window
        -  context window 크기. 중심단어를 기준으로 좌우 몇개의 단어를 확인하는지 크기. 기본값: 5
    - min_count
        - 이 설정보다 낮은 빈도로 등장하는 단어는 무시한다. 데이터 노이즈를 줄이는데 도움이된다. 기본값: 5
    - sg
        - 모델 아키텍처 결정.
        - `0`: CBOW, `1`: Skip-gram. 기본값: 0
    - epochs
        - epochs 수 설정. 기본값: 5
    - alpha
        - initial leaning rate. 기본값: 0.025
    - min_alpha
        - 최소 learning rate. 기본값: 0.0001
        - epoch 마다 learning rate를 alpha 에서 min_alpha 까지 선형적으로 줄여나간다.
    - workers
        -  사용 Thread 수. (= cpu의 개수) 기본값: 3

## 학습(Train)
1. Word2Vec 의 initializer에 sentences를 넣어 한번에 학습한다.
2. Word2Vec 클래스에 학습 설정을 하고 `train()` 메소드를 이용해 학습한다.
    - epoch 단위로 작업을 할 경우 사용

epoch을 내가 지정할 수 있다!

In [3]:
# 샘플 텍스트 데이터
sentences = [
    "Natural language processing is an exciting field of study",
    "Word embeddings are a type of word representation",
    "Gensim is a powerful library for text processing",
    "Word2Vec creates vector representations of words", 
    "Gensim runs on Linux, Windows and OS X, as well as any other platform that supports Python and NumPy."
    "All Gensim source code is hosted on Github under the GNU LGPL license, maintained by its open source community.",
    "For commercial arrangements, see Business Support.",
    "Gensim can process arbitrarily large corpora, using data-streamed algorithms.",
    "There are no \"dataset must fit in RAM\" limitations."
]

토큰화하는 건 우리가 해야 함. 함수를 정의합니다.

In [4]:
import re
import nltk

def tokenizer(docs):
    # 소문자로 모두 변환
    # 알파벳, 숫자, _,를 제외한 모든 문자들을 제거
    # 단어(어절) 단위 토큰화
    return [nltk.word_tokenize(re.sub(r"[^\w\s]", "", doc.lower())) for doc in sentences]

# [^\w\s] - \w(문자) '또는' \s(공백 문자)를 제외한 나머지(^)들을 찾아라.
# 공백 문자 \s를 포함하지 않으면: 공백 문자도 없어져버림.


In [5]:
import re
# [re.sub(r"[^\w]", "", doc.lower()) for doc in sentences] # 문장 전처리
 # 토큰화
tokens = tokenizer(sentences)
tokens

[['natural',
  'language',
  'processing',
  'is',
  'an',
  'exciting',
  'field',
  'of',
  'study'],
 ['word', 'embeddings', 'are', 'a', 'type', 'of', 'word', 'representation'],
 ['gensim', 'is', 'a', 'powerful', 'library', 'for', 'text', 'processing'],
 ['word2vec', 'creates', 'vector', 'representations', 'of', 'words'],
 ['gensim',
  'runs',
  'on',
  'linux',
  'windows',
  'and',
  'os',
  'x',
  'as',
  'well',
  'as',
  'any',
  'other',
  'platform',
  'that',
  'supports',
  'python',
  'and',
  'numpyall',
  'gensim',
  'source',
  'code',
  'is',
  'hosted',
  'on',
  'github',
  'under',
  'the',
  'gnu',
  'lgpl',
  'license',
  'maintained',
  'by',
  'its',
  'open',
  'source',
  'community'],
 ['for', 'commercial', 'arrangements', 'see', 'business', 'support'],
 ['gensim',
  'can',
  'process',
  'arbitrarily',
  'large',
  'corpora',
  'using',
  'datastreamed',
  'algorithms'],
 ['there', 'are', 'no', 'dataset', 'must', 'fit', 'in', 'ram', 'limitations']]

문장별로 토큰 리스트를 저장한 게 리스트로 묶여있는 거임

In [6]:
# 학습 - train
# Word2Vec 객체를 생성: 학습 데이터, epoch을 지정 -> 객체 생성할 때 모델을 학습까지! 하긋ㅂ 결과 모델을 반환한다. 
from gensim.models import Word2Vec
model1 = Word2Vec(
    sentences= tokens, # 학습 데이터
    vector_size=10, # embedding vector의 차원 (한 개 단어에서 몇 개 feature를 추출할지)
    window=2, # window size 설정. 주변 단어의 개수
    min_count=1, # 최소 출연 빈도수. 얘가 있어야 vocab을 제대로 만들어요!
    epochs=10,
    workers=3 # 병렬 처리 개수. cpu 개수에 따라 달라진다.
)

In [7]:
import os
os.cpu_count() # 현재 컴퓨터에서 병렬로 처리할 수 있는 최대 개수, 코어가 몇 개 있는지
# workers에 os.cpu_count()를 넣으면 -> 컴퓨터 자원을 모두 다 쓰겠다.

8

In [8]:
# 학습데이터, epoch을 설정하지 않음 => 학습 안 된 모델을 반환
model2 = Word2Vec(vector_size=10, window=2, min_count=1, workers=os.cpu_count())
# model에 vocab을 설정.
model2.build_vocab(tokens)
# 학습
epoch=10

# 1 epoch 단위로 작성하는 경우
for e in range(epoch):
    model2.train(
        tokens, # 학습 데이터
        total_examples=model2.corpus_count, # 학습 데이터(문서) 개수
        epochs=1, 
        compute_loss = True # 학습이 끝나면 loss 계산
        # com_loss=True 해줘야 loss를 가져올 수 있다. 
    )
loss = model2.get_latest_training_loss() # train()시 계산된 loss를 조회
print(f"{e} epoch loss: {loss}")
'''
model2.train(
        tokens, # 학습 데이터
        total_examples=model2.corpus_count, # 학습 데이터(문서) 개수
        epochs=10, 
    )
위와 같은 결과임.
'''


9 epoch loss: 156.24295043945312


'\nmodel2.train(\n        tokens, # 학습 데이터\n        total_examples=model2.corpus_count, # 학습 데이터(문서) 개수\n        epochs=10, \n    )\n위와 같은 결과임.\n'

## 학습 후 결과 조회

- **KeyedVectors 조회**
    - KeyedVectors는 **단어와 vector를 매핑한 객체**로 embedding vector를 이용한 다양한 조회를 지원한다.
    - model.wv 로 조회해서 사용.
- **Embedding Vector 조회**
  - model.wv.vectors
- **단어 목록 조회**
    - model.wv.index_to_key, model.wv.key_to_index
- **단어 벡터 조회**
    - model.wv[word]: 특정 단어의 vector반환
- **Vocab에 대상 단어가 있는지 확인**
    - "대상단어" in model.wv
- **유사단어들 찾기**
    - model.wv.most_similar(word)
- **단어간 유사도 비교**
    - model.wv.similarity(word1, word2)
- 유사도를 계산할 때 **코사인 유사도(Cosine Similarity)** 를 사용한다.

> # 코사인 유사도
> - 두 벡터 간의 유사성을 측정하는 중요한 방법 중 하나.
> - 코사인 유사도는 두 벡터 간의 코사인 각도를 이용하여 유사도를 계산한다. 이때 벡터의 **크기는 결과에 영향을 미치지 않고, 오직 방향만이 중요**하다.
> ## 공식
> 
> $$ similarity = cos(\theta) = \frac{A⋅B}{||A||\ ||B||} = \frac{\sum_{i=1}^{n}{A_i×B_i}}{\sqrt{\sum_{i=1}^{n}(A_i)^2}×\sqrt{\sum_{i=1}^{n}(B_i)^2}} $$
> 
> ## 결과 해석
> 
> - **값의 범위**: -1에서 1 사이의 값을 가집니다
>   - 1: 두 벡터가 완전히 동일한 방향 (0도의 cosine 값)
>   - 0: 두 벡터가 직교 (90도의 cosine 값)
>   - -1: 두 벡터가 정반대 방향 (180도의 cosine 값)
> 
> ![cosine_similarity](figures/gensim_consin_sim.png)
>
> ## Python 코사인 유사도 계산
> ```python
> from numpy import dot
> from numpy.linalg import norm
> 
> def cosine_similarity(A, B):
>     return dot(A, B)/(norm(A)*norm(B))
> ```



값의 크기가 아닌 **방향**

In [9]:
model1.wv # KeyedVectors -> 토큰 - embedding vector

In [10]:
model1.wv.index_to_key # vocab: token_id -> token (단어)

['gensim',
 'is',
 'of',
 'for',
 'processing',
 'source',
 'as',
 'on',
 'word',
 'are',
 'a',
 'and',
 'words',
 'runs',
 'linux',
 'os',
 'windows',
 'vector',
 'x',
 'well',
 'any',
 'representations',
 'limitations',
 'creates',
 'word2vec',
 'text',
 'platform',
 'library',
 'powerful',
 'representation',
 'type',
 'embeddings',
 'study',
 'field',
 'exciting',
 'an',
 'language',
 'other',
 'that',
 'ram',
 'see',
 'support',
 'can',
 'process',
 'arbitrarily',
 'large',
 'corpora',
 'using',
 'datastreamed',
 'algorithms',
 'there',
 'no',
 'dataset',
 'must',
 'fit',
 'in',
 'business',
 'arrangements',
 'supports',
 'commercial',
 'python',
 'numpyall',
 'code',
 'hosted',
 'github',
 'under',
 'the',
 'gnu',
 'lgpl',
 'license',
 'maintained',
 'by',
 'its',
 'open',
 'community',
 'natural']

In [11]:
model1.wv.key_to_index

{'gensim': 0,
 'is': 1,
 'of': 2,
 'for': 3,
 'processing': 4,
 'source': 5,
 'as': 6,
 'on': 7,
 'word': 8,
 'are': 9,
 'a': 10,
 'and': 11,
 'words': 12,
 'runs': 13,
 'linux': 14,
 'os': 15,
 'windows': 16,
 'vector': 17,
 'x': 18,
 'well': 19,
 'any': 20,
 'representations': 21,
 'limitations': 22,
 'creates': 23,
 'word2vec': 24,
 'text': 25,
 'platform': 26,
 'library': 27,
 'powerful': 28,
 'representation': 29,
 'type': 30,
 'embeddings': 31,
 'study': 32,
 'field': 33,
 'exciting': 34,
 'an': 35,
 'language': 36,
 'other': 37,
 'that': 38,
 'ram': 39,
 'see': 40,
 'support': 41,
 'can': 42,
 'process': 43,
 'arbitrarily': 44,
 'large': 45,
 'corpora': 46,
 'using': 47,
 'datastreamed': 48,
 'algorithms': 49,
 'there': 50,
 'no': 51,
 'dataset': 52,
 'must': 53,
 'fit': 54,
 'in': 55,
 'business': 56,
 'arrangements': 57,
 'supports': 58,
 'commercial': 59,
 'python': 60,
 'numpyall': 61,
 'code': 62,
 'hosted': 63,
 'github': 64,
 'under': 65,
 'the': 66,
 'gnu': 67,
 'lgpl': 

In [12]:
model1.wv['gensim']

array([-0.00530078,  0.00215998,  0.0511257 ,  0.08994552, -0.0929675 ,
       -0.07124555,  0.06510045,  0.08978292, -0.05092729, -0.0380263 ],
      dtype=float32)

In [13]:
# in 연산자 -> vocab에 특정 단어가 있는지 여부
# model1.wv['안녕'] # '안녕'은 토큰에 없음 -> KeyError
'안녕' in model1.wv  # False
'gensim' in model1.wv  # True

True

In [14]:
word = '안녕'
if word in model1.wv:
    print(model1.wv[word])
else:
    print("없는 단어입니다.")

없는 단어입니다.


In [15]:
# 유사도 검사
model1.wv.most_similar("gensim") # 'gensim'과 가장 유사한 단어를 순서대로 10개를 반환.
# (단어, 유사도) 유사도 -1 ~ 1 (1에 가까울 수록 유사, -1에 가까울 수록 반대)

[('numpyall', 0.7189716696739197),
 ('study', 0.7138967514038086),
 ('that', 0.6710396409034729),
 ('can', 0.6415793299674988),
 ('datastreamed', 0.5988761186599731),
 ('is', 0.5452057123184204),
 ('other', 0.5355265140533447),
 ('maintained', 0.5195195078849792),
 ('representations', 0.5130788087844849),
 ('in', 0.507987380027771)]

In [16]:
model1.wv.most_similar("gensim", topn=3) # 상위 3개만 볼래요~

[('numpyall', 0.7189716696739197),
 ('study', 0.7138967514038086),
 ('that', 0.6710396409034729)]

In [17]:
# 단어 간의 유사도
model1.wv.similarity("gensim", "study") 

0.71389675

## 모델 저장 및 로딩

### 모델 저장, 로딩
- `model.save('저장파일 경로')`
  - gensim 자체 포맷으로 저장된다.
- `gensim.models.Word2Vec.load('저장파일 경로')`
  - `model.save()`로 저장된 모델을 Loading한다.

In [19]:
import os
os.makedirs("saved_models", exist_ok=True)
model1.save("saved_models/w2v_model.model")

In [20]:
from gensim.models import Word2Vec
load_model = Word2Vec.load("saved_models/w2v_model.model")

In [21]:
load_model.wv.most_similar("gensim")

[('numpyall', 0.7189716696739197),
 ('study', 0.7138967514038086),
 ('that', 0.6710396409034729),
 ('can', 0.6415793299674988),
 ('datastreamed', 0.5988761186599731),
 ('is', 0.5452057123184204),
 ('other', 0.5355265140533447),
 ('maintained', 0.5195195078849792),
 ('representations', 0.5130788087844849),
 ('in', 0.507987380027771)]

위와 같은 결과가 나옴! 제대로 로드 되었다는 뜻이다

### Word Embedding Vector만 저장 및 로드
- `KeyedVectors` 를 이용해 저장한다.
    - `model.wv.save_word2vec_format('저장경로', binary=True|False)`
        - binary=True: binary 파일로 저장한다. 용량이 작은 대신 내용확인이 안된다.
        - binary=False: csv(공백구분자) 형식 text로 저장한다. 내용을 확인할 수있지만 파일용량이 크다.
- `KeyedVectors.load_word2vec_format("저장경로", binary=True|False)`
    - 저장시 binary에 맞춰 읽는다.

In [23]:
model1.wv # KeyedVectors가 반환된다

In [24]:
model1.wv.save_word2vec_format("saved_models/keyedvector.bin", binary=True)
model1.wv.save_word2vec_format("saved_models/keyedvector.csv", binary=False)

In [25]:
from gensim.models import KeyedVectors
load_wv_bin = KeyedVectors.load_word2vec_format("saved_models/keyedvector.bin", binary=True)
load_wv_csv = KeyedVectors.load_word2vec_format("saved_models/keyedvector.csv", binary=False)

In [27]:
load_wv_bin.most_similar('gensim')

[('numpyall', 0.7189716696739197),
 ('study', 0.7138967514038086),
 ('that', 0.6710396409034729),
 ('can', 0.6415793299674988),
 ('datastreamed', 0.5988761186599731),
 ('is', 0.5452057123184204),
 ('other', 0.5355265140533447),
 ('maintained', 0.5195195078849792),
 ('representations', 0.5130788087844849),
 ('in', 0.507987380027771)]

잘 읽어와졌다.. 👍🏼

transfer learning: 이미 만들어진 것을 가져다 쓴다. 거기다 필요하면 내용 추가

## Pretrained 모델 사용하기
- https://github.com/Kyubyong/wordvectors

강사님이 주신 ko_new 파일을 이용합니당.

In [28]:
from zipfile import ZipFile
with ZipFile("ko_new.zip") as zf: # 압축풀 파일 경로
    zf.extractall("saved_models")

extract: 개별 파일을 하나씩 푼다.

extractall: 모든 파일을 다 푼다.

In [29]:
ko_wv = KeyedVectors.load_word2vec_format(
    "saved_models/ko_new.txt",
    binary=False,
    encoding='utf-8'
)

In [30]:
word = "저녁"
result = ko_wv.most_similar(word, topn=3)
print(result)

[('아침', 0.8432559967041016), ('밤', 0.7863156795501709), ('새벽', 0.7520349025726318)]


In [31]:
word = "호랑이"
result = ko_wv.most_similar(word, topn=3)
print(result)

[('코끼리', 0.7047041654586792), ('표범', 0.6993285417556763), ('멧돼지', 0.6985809803009033)]


In [35]:
word = "푸딩"
result = ko_wv.most_similar(word, topn=3)
print(result)

[('비스킷', 0.7836082577705383), ('시럽', 0.7781529426574707), ('초콜릿', 0.7692068815231323)]


# 빅카인즈 뉴스 데이터를 이용한 Word2Vec 학습
- 빅카인즈
    - 한국언론진흥재단에서 운영하는 뉴스빅데이터 분석 서비스 사이트
- 빅카인즈에서 특정 분야의 기사들을 수집해서 학습시킨다.
    - https://www.bigkinds.or.kr/
    - 회원가입 (구글, 네이버, 카카오 계정으로 가입 가능) 후 로그인
    - 뉴스분석 > 뉴스검색$\cdot$분석 클릭
    ![word2vec_bigkinds1.png](figures/word2vec_bigkinds1.png)
    - 기간, 언론사, 분류, 상세검색 등 검색 조건입력 후 조회
    ![word2vec_bigkinds1.png](figures/word2vec_bigkinds2.png)
    - 결과 다운로드
        - step3 분석결과및 시각화 -> 맨 아래 `엑셀다운로드` 클릭 
    ![word2vec_bigkinds1.png](figures/word2vec_bigkinds3.png)

In [43]:
%pip install openpyxl

Note: you may need to restart the kernel to use updated packages.


In [47]:
import pandas as pd
df = pd.read_excel("NewsResult_20250223-20250523.xlsx")

c:\Anaconda3\envs\dl\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [50]:
df = df[['제목', '본문']]
df

,제목,본문
0,"“내가 재림예수다” 허경영, 구속 송치 ‘대천사’ 칭호 주고 1억 원 챙겨",고가의 영성 상품을 판매하고 신도들을 추행한 혐의를 받는 허경영 국가혁명당 명예 대...
1,한미 관세 2차 실무협의 마무리 다음 협의는 대선 이후,미국의 상호관세 및 품목관세 문제를 협의하기 위한 한미 국장급 2차 실무 협의가 마...
2,"박명수 경기도의원, “강산 4번 바뀌는 동안 안성은 바뀐 것 하나 없다”","경기도의회 박명수 의원(국민의힘, 안성2)은 21일(화) 안성상담소에서 수자원본부로..."
3,"LS전선 구본규 대표, 베트남서 해저망 희토류 사업 공략",아시아투데이 김한슬 기자 = 구본규 LS전선 대표가 베트남 기업 및 정부 관계자들을...
4,1000만원 내면 ‘대통령대리’ 임명에 체포 면제? ‘기막힌 허경영의 사기행각’,영적 능력이 있다며 최대 1억원에 달하는 영성상품을 판매하고 신도들을 강제로 추행한...
...,...,...
3814,아파트 분양가 이래서 비쌌나 한샘 리바트 등 빌트인 가구 담합,"한샘, 현대리바트 등 13개 가구업체가 반도건설이 발주한 아파트 특판가구 입찰에서 ..."
3815,"‘분식회계 채용비리’ 하성용 전 KAI 대표, 징역형 집행유예 확정",수천억원대 분식회계와 채용 비리 의혹 등으로 재판에 넘겨진 하성용 전 한국항공우주산...
3816,“원칙과 소신이 중요” ‘계엄’과 ‘탄핵’ 겪은 한 ‘공인’의 조언,대한민국이 좀처럼 혼돈에서 벗어나지 못하고 있다. 헌법재판소의 윤석열 대통령 탄핵 ...
3817,트럼프와 머스크의 ‘해고 칼춤’ 미 공무원 1만명 일자리 잃는다,찍힌 기관들 사실상 폐쇄 절차 사법부 ‘일시 중단’ 명령도 무시\n\n\n\n[주간...


## TODO

In [51]:
# 문서: 개별 뉴스기사 (제목 + 본문)
# 전처리 + 토큰화 - 소문자 통일, 형태소 단위 토큰화

# gensim을 이용해서 embedding 모델 학습

# 결과 확인 -> 유사도 검사사